<a href="https://colab.research.google.com/github/JavierPerez21/QHack2022/blob/master/Coding_Challenges/pennylane101_500_BitflipErrorCode_template/pennylane101_500_BitFlipError.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install pennylane

In [ ]:
import pennylane as qml
from pennylane import numpy as np

The goal of this challenge is to detect the probabilities that a bit flip error has occurred in a given circuit. For this, assume we begin with the state $|\psi\rangle = \alpha |0\rangle + \sqrt{1-\alpha²} |1\rangle$ encoded in a logical three-qubit state. Then we need to complete the function `circuit()` to perform some preprocessing on $|\psi\rangle$ perform the random BitFlip and some post processing on the resulting state. Then, we need to complete the function `error_wire()` that will take as an input the output of a circuit and output the probability of a bitflip error not having ocurred and the probability of an error happening in each wire.

To solve this challenge, since we have knowledge of which wire is being tampered, we can use the correction approach shown [here](https://quantumcomputinguk.org/tutorials/quantum-error-correction-bit-flip-code-in-qiskit#:~:text=What%20is%20a%20Bit%20Flip,qubits%20to%20correct%201%20qubit.).

In [ ]:
def density_matrix(alpha):
    """Creates a density matrix from a pure state."""
    # DO NOT MODIFY anything in this code block
    psi = alpha * np.array([1, 0], dtype=float) + np.sqrt(1 - alpha**2) * np.array(
        [0, 1], dtype=float
    )
    psi = np.kron(psi, np.array([1, 0, 0, 0], dtype=float))
    return np.outer(psi, np.conj(psi))

In [ ]:
dev = qml.device("default.mixed", wires=3)


@qml.qnode(dev)
def circuit(p, alpha, tampered_wire):
    """A quantum circuit that will be able to identify bitflip errors.

    DO NOT MODIFY any already-written lines in this function.

    Args:
        p (float): The bit flip probability
        alpha (float): The parameter used to calculate `density_matrix(alpha)`
        tampered_wire (int): The wire that may or may not be flipped (zero-index)

    Returns:
        Some expectation value, state, probs, ... you decide!
    """

    qml.QubitDensityMatrix(density_matrix(alpha), wires=[0, 1, 2])

    # QHACK #
    # Preprocessing
    others = [i for i in [0, 1, 2] if i != tampered_wire]
    qml.CNOT(wires=[tampered_wire, others[0]])
    qml.CNOT(wires=[tampered_wire, others[1]])
    # QHACK #

    qml.BitFlip(p, wires=int(tampered_wire))

    # QHACK #  
    # Post-processing
    qml.CNOT(wires=[tampered_wire, others[0]])
    qml.CNOT(wires=[tampered_wire, others[1]])
    qml.Toffoli(wires=[others[0], others[1], tampered_wire])
    return qml.state()
    # QHACK #

def error_wire(circuit_output):
    """Function that returns an error readout.

    Args:
        - circuit_output (?): the output of the `circuit` function.

    Returns:
        - (np.ndarray): a length-4 array that reveals the statistics of the
        error channel. It should display your algorithm's statistical prediction for
        whether an error occurred on wire `k` (k in {1,2,3}). The zeroth element represents
        the probability that a bitflip error does not occur.

        e.g., [0.28, 0.0, 0.72, 0.0] means a 28% chance no bitflip error occurs, but if one
        does occur it occurs on qubit #2 with a 72% chance.
    """

    # QHACK #
    probs = abs(np.array([circuit_output[i][i] for i in range(0, len(circuit_output))]))
    p_, p0, p1, p2 = 0, 0, 0, 0
    p_ = probs[0] + probs[4]
    if probs[7] != 0:
        p0 = probs[7] + probs[3]
    if probs[5] != 0:
        p1 = probs[5] + probs[3]
    if probs[6] != 0:
        p2 = probs[6] + probs[3]

    # QHACK #
    return [p_, p0, p1, p2]

Testing our solution

In [ ]:
prob = np.random.rand()
alpha = np.random.rand()
idx = np.random.randint(0, 3)
inputs = np.array([prob, alpha, idx], dtype=float)
p, alpha, tampered_wire = inputs[0], inputs[1], int(inputs[2])

error_readout = np.zeros(4, dtype=float)
circuit_output = circuit(p, alpha, tampered_wire)
error_readout = error_wire(circuit_output)
print(f"Probability of error not happening -> Predicted: {np.round(error_readout[0]*100, 2)}% vs Expected: {np.round((1 - inputs[0])*100,2)}%")
for i in range(0, 3):
  if inputs[-1] == i:
      prob = np.round(inputs[0]*100, 2)
  else:
      prob = 0
  print(f"Probability of error happening on wire {i}: {np.round(error_readout[i+1]*100, 2)}% vs Expected: {prob}%")

Now, let's see why our `error_wire` function works.

First, notice that our `circuit` function makes use of the `density_matrix` function and so the input to our preprocessing and post processing is a matrix $M$ given by:

$$
 M = |\psi\rangle \langle \psi | \;\;\; where  \;\;\; M_{i,j} = |\psi_i\rangle \langle \psi_j |
$$

Hence, the output of our circuit is another matrix $A$ of the same shape as $M$.

Let's analyze the outputs along the diagonal of a few of these circuits.

In [ ]:
prob = np.random.rand()
alpha = np.random.rand()
for idx in [0, 1, 2]:  
  inputs = np.array([prob, alpha, idx], dtype=float)
  p, alpha, tampered_wire = inputs[0], inputs[1], int(inputs[2])
  circuit_output = circuit(p, alpha, tampered_wire)
  diag = np.array([np.round(circuit_output[i][i].real, 4) for i in range(0, len(circuit_output))])
  diag = diag.reshape(2, 2, 2)
  print(f"Tampering wire {idx}")
  expected_output = [np.round((1 - inputs[0]), 4), 0, 0, 0]
  expected_output[idx+1] = np.round(inputs[0], 4)
  print(f"Expected output = {expected_output}")
  print(" Values for eachs tate")
  for i in [0, 1]:
    for j in [0, 1]:
      for k in [0, 1]:
        out = diag[i][j][k]
        print(f"  State|{i}{j}{k}> --> {out}")

From this it is easy to see that the null probability (probability of an error not happening) is always given by the sum of values corresponding to states $|000\rangle$ and $|100\rangle$. We can also observe that the probabilities of the error happening on qubits 0, 1 or 2 are given by the sum of the value corresponding to state $||011\rangle$ and the value corresponding to states $|111\rangle$, $|101\rangle$ and $|1101\rangle$ respectively, which is exactly what we do in our solution.

```
probs = abs(np.array([circuit_output[i][i] for i in range(0, len(circuit_output))]))
p_, p0, p1, p2 = 0, 0, 0, 0
p_ = probs[0] + probs[4]
if probs[7] != 0:
    p0 = probs[7] + probs[3]
if probs[5] != 0:
    p1 = probs[5] + probs[3]
if probs[6] != 0:
    p2 = probs[6] + probs[3]
```